In [ ]:
import os, sys 
#to be able to interact with Google Drive's operating system
from google.colab import drive 
#drive is a module that allows us use Python to interact with google drive
drive.mount('/content/gdrive') 
#mounting google drive allows us to work with its contents
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)
#The last three lines are what changes the path of the file.c

Mounted at /content/gdrive


Install the required package

In [ ]:
!pip install tigramite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.1
    Uninstalling llvmlite-0.39.1:
      Successfully uninstalled llvmlite-0.39.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: numba
    Fo

Import requied libraries

In [ ]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
#import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

/usr/local/lib/python3.8/dist-packages/tigramite/plotting.py:26: UserWarning: [Errno 2] No such file or directory: '/usr/local/lib/python3.8/dist-packages/tigramite/../versions.py'
  warnings.warn(str(e))
/usr/local/lib/python3.8/dist-packages/tigramite/independence_tests/gpdc.py:27: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
/usr/local/lib/python3.8/dist-packages/tigramite/independence_tests/gpdc_torch.py:33: UserWarning: No module named 'dcor'
  warnings.warn(str(e))


Upload the dataset we are interested to find the lagged causal links

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 8.4.csv to 8.4.csv


In [ ]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['8.4.csv']))# Dataset is now stored in a Pandas Dataframe

In [ ]:
df=df.to_numpy()
print(df)

[[1.44525442 1.48616357 3.85930566 ... 1.74997337 0.92289849 1.64133233]
 [1.21034239 1.21664694 4.34039276 ... 0.94385575 1.56740765 2.09380629]
 [1.6201918  1.28164556 4.19791397 ... 1.64005272 1.55373346 1.86700844]
 ...
 [0.6604136  2.20501344 7.93984376 ... 7.79686836 5.32864923 4.91237873]
 [0.5379207  1.50437159 7.06520748 ... 8.31661265 4.70672634 4.46542932]
 [0.70641374 1.24015198 7.32849856 ... 9.54590127 5.81298936 5.17626493]]


In [ ]:
# Initialize dataframe object, specify time axis and variable names
var_names = [r'$X^1$', r'$X^2$', r'$X^3$', r'$X^4$', r'$X^5$', r'$X^6$', r'$X^7$', r'$X^8$']
dataframe = pp.DataFrame(df, 
                         datatime = np.arange(len(df)), 
                         var_names=var_names)

Perform the test using the parameters: significance level of the CI tests, minimum and maximum period of lags. Record the lagged links for the further steps.

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=0, tau_max=4, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.043

    Variable $X^2$ has 0 link(s):

    Variable $X^3$ has 2 link(s):
        ($X^2$ -2): pval = 0.00000 | val =  0.030
        ($X^3$ -3): pval = 0.03000 | val =  0.012

    Variable $X^4$ has 1 link(s):
        ($X^3$  0): pval = 0.00000 | val =  0.025

    Variable $X^5$ has 1 link(s):
        ($X^6$  0): pval = 0.00000 | val =  0.022 | unoriented link

    Variable $X^6$ has 1 link(s):
        ($X^5$  0): pval = 0.00000 | val =  0.022 | unoriented link

    Variable $X^7$ has 1 link(s):
        ($X^6$ -1): pval = 0.00000 | val =  0.070

    Variable $X^8$ has 1 link(s):
        ($X^7$  0): pval = 0.00000 | val =  0.079


133 mins

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=1, tau_max=4, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.046

    Variable $X^2$ has 0 link(s):

    Variable $X^3$ has 3 link(s):
        ($X^2$ -2): pval = 0.00000 | val =  0.065
        ($X^3$ -3): pval = 0.00000 | val =  0.015
        ($X^1$ -1): pval = 0.03100 | val =  0.010

    Variable $X^4$ has 2 link(s):
        ($X^2$ -2): pval = 0.00000 | val =  0.071
        ($X^3$ -3): pval = 0.00400 | val =  0.016

    Variable $X^5$ has 0 link(s):

    Variable $X^6$ has 1 link(s):
        ($X^5$ -3): pval = 0.02300 | val =  0.012

    Variable $X^7$ has 1 link(s):
        ($X^6$ -1): pval = 0.00000 | val =  0.093

    Variable $X^8$ has 1 link(s):
        ($X^6$ -1): pval = 0.00000 | val =  0.057


57.5 mins